# Day024-Pandas综合练习项目-自动化EDA之dataprep

## 虚拟环境的使用

1. conda env list  : 查看当前系统下的虚拟环境有哪些
      
       conda env list
        # conda environments:
        #
        base                     D:\Anaconda3
        dataprepeda              D:\Anaconda3\envs\dataprepeda\
        
2. conda create -n dataprep python=3.10 # 创建一个名为dataprep的虚拟环境,其中python版本为3,10

3. conda activate dataprep  # 激活虚拟环境

4. conda list # 查看当前虚拟环境中,都安装了哪些包

5. conda install -c conda-forge dataprep # 安装dataprep这个包

### 第三方虚拟环境

- dataprep.zip 564M: 包含dataprep宏包的
- datapreptest.zip 47M: 仅有一个Python3.10环境的

使用步骤:

1. 打开anaconda prompt:
2. 激活虚拟环境: conda activate F:\dataprep      # 后面的路径,使用你实际的路径
3. 可以输入ipython执行一下版本检测,这里用的是3.10.15
4. 开始进行自动化EDA操作:

    方法1: 在当前环境输入jupyter notebook 打开jupyter notebook开始工作
    
    方法2: 输入ipython进入交互式编程环境,编写代码
    
    方法3: 在任意位置执行: F:\dataprep\python.exe C:\Users\Matri\Desktop\test.py

## 补充关于Pyecharts自定义地图实现

In [24]:
import json
from pyecharts.charts import Geo
from pyecharts import options as opts
# 读取 GeoJSON 文件
with open('./data/map (4).geojson', 'r', encoding='utf-8') as f:
    map_json1 = json.load(f)

In [25]:
geo = Geo(init_opts=opts.InitOpts(theme='light',
                                      width='1000px',
                                      height='600px'))
# 添加自定义地图
geo.add_js_funcs(f"echarts.registerMap('河南师范大学', {map_json1});")
# 引用添加的地图
geo.add_schema(maptype="河南师范大学")
geo.render_notebook()

In [19]:
from pyecharts.charts import Map
maps = Map(init_opts=opts.InitOpts(theme='light',
                                      width='1000px',
                                      height='600px'))
maps.add_js_funcs(f"echarts.registerMap('河南师范大学', {map_json1});")
# 引用添加的地图
maps.add("河师大",[('河南师范大学',100)],maptype='河南师范大学')
maps.render_notebook()

1. 第一步到https://geojson.io/网站中,圈出你要的地图
2. 到pyehcarts使用maps.add_js_funcs(f"echarts.registerMap('河南师范大学', {map_json});") 注册地图文件

### 省份地图-精确到区

In [106]:
with open('./data/map (1).geojson', 'r', encoding='utf-8') as f:
    map_json = json.load(f)

In [107]:
from pyecharts.charts import Map
maps = Map(init_opts=opts.InitOpts(theme='light',
                                      width='1000px',
                                      height='600px'))
maps.add_js_funcs(f"echarts.registerMap('河南-区', {map_json});")
# 引用添加的地图
maps.add("河师大",[('红旗区',100)],maptype='河南-区')
maps.render()

'C:\\Users\\Matri\\Desktop\\河南师范大学\\综合实训课程1\\Pandas数据分析练习题\\render.html'

## 多个省份数据拼接

- 山河四省: 河南-河北-山东-山西

In [108]:
with open('./data/maps/河南省.json', 'r', encoding='utf-8') as f:
    map_json = json.load(f)
from pyecharts.charts import Map
maps = Map(init_opts=opts.InitOpts(theme='light',
                                      width='1000px',
                                      height='600px'))
maps.add_js_funcs(f"echarts.registerMap('河南T', {map_json});")
# 引用添加的地图
maps.add("河师大",[('河南省',100)],maptype='河南T')
maps.render()
maps.render_notebook()

In [109]:
with open('./data/河南省-区.json', 'r', encoding='utf-8') as f:
    map_json_t = json.load(f)

In [110]:
map_json_t['features'][0]['properties']

{'adcode': 410102,
 'name': '中原区',
 'center': [113.611576, 34.748286],
 'centroid': [113.546928, 34.780838],
 'childrenNum': 0,
 'level': 'district',
 'parent': {'adcode': 410100},
 'subFeatureIndex': 0,
 'acroutes': [100000, 410000, 410100]}

In [111]:
len(map_json_t['features'])

157

In [112]:
import os
filenames = os.listdir('./data/maps/')
shss = {'type': 'FeatureCollection','features':[]}
for i in filenames:
    with open(f"./data/maps/{i}",'r',encoding='utf-8') as f:
        data = json.loads(f.read())
        shss['features'].append(data)

In [113]:
import numpy as np
# json地图数据的构建
target_data = """{
        "type": "FeatureCollection",
        "features": []}"""
target_data = json.loads(target_data)
# 将云川渝的数据节点传入target_data，并加入area节点
list_region = os.listdir('./data/maps/')
name_list = []
for lr in list_region:
    with open(f"./data/maps/{lr}", 'r', encoding='utf-8') as f:
        temp = json.load(f)
        temp = temp['features']
        for tp in temp:
            name = tp['properties']['name']
            # 将area加入tp节点并传入target_data
            tp['properties']['area'] = '山河四省'
            target_data['features'].append(tp)
            name_list.append((name, np.random.randint(100, 5000)))

In [114]:
from pyecharts.charts import Map
maps = Map(init_opts=opts.InitOpts(theme='light',
                                      width='1000px',
                                      height='600px'))
maps.add_js_funcs(f"echarts.registerMap('山河四省', {target_data});")
# 引用添加的地图
maps.add("河师大",[('河南省',100)],maptype='山河四省')
maps.render()
maps.render_notebook()

任务1: 绘制 江浙沪 地区地图, 云贵川渝地图
    
任务2: 绘制 河南师范大学三个校区

## 任务1: 绘制 江浙沪 地区地图, 云贵川渝地图

第一步: 找到云南,贵州,四川,重庆地图

    1. 地图怎么找: datav组件,找出这些地图的数据

In [119]:
import numpy as np
# json地图数据的构建
target_data = """{
        "type": "FeatureCollection",
        "features": []}"""
target_data = json.loads(target_data)
# 将云川渝的数据节点传入target_data，并加入area节点
name_l = '江浙沪'
list_region = os.listdir(f'./data/{name_l}/')
name_list = []
for lr in list_region:
    with open(f"./data/{name_l}/{lr}", 'r', encoding='utf-8') as f:
        temp = json.load(f)
        temp = temp['features']
        for tp in temp:
            name = tp['properties']['name']
            # 将area加入tp节点并传入target_data
            tp['properties']['area'] = '云贵川渝'
            target_data['features'].append(tp)
            name_list.append((name, np.random.randint(100, 5000)))

In [124]:
from pyecharts.charts import Map
maps = Map(init_opts=opts.InitOpts(theme='light',
                                      width='1000px',
                                      height='600px'))
maps.add_js_funcs(f"echarts.registerMap('江浙沪', {target_data});")
# 引用添加的地图
maps.add("",[('江苏省',100),('上海市',40)],maptype='江浙沪',is_map_symbol_show=False,
        label_opts=opts.LabelOpts(is_show=False)
        )
maps.set_global_opts(visualmap_opts=opts.VisualMapOpts())
maps.render()
maps.render_notebook()

In [126]:
import json
from pyecharts.charts import Geo
from pyecharts import options as opts
# 读取 GeoJSON 文件
with open('./data/河南师范大学.geojson', 'r', encoding='utf-8') as f:
    map_json1 = json.load(f)
geo = Geo(init_opts=opts.InitOpts(theme='light',
                                      width='1000px',
                                      height='600px'))
# 添加自定义地图
geo.add_js_funcs(f"echarts.registerMap('河南师范大学', {map_json1});")
# 引用添加的地图
geo.add_schema(maptype="河南师范大学")
geo.render_notebook()

## 虚拟环境管理

1. conda env list  : 查看当前系统下的虚拟环境有哪些
      
       conda env list
        # conda environments:
        #
        base                     D:\Anaconda3
        dataprepeda              D:\Anaconda3\envs\dataprepeda\
        
2. conda create -n dataprep python=3.10 # 创建一个名为dataprep的虚拟环境,其中python版本为3,10

3. conda activate dataprep  # 激活虚拟环境

4. conda list # 查看当前虚拟环境中,都安装了哪些包

5. conda install -c conda-forge dataprep # 安装dataprep这个包



6.  conda env remove --name datapreptest  # 删除虚拟环境
7.  conda list --name dataprep  # 查看指定虚拟环境中安装的包有哪些?